In [3]:
from ultralytics import YOLO
import cv2
import numpy as np

# Load model
model = YOLO('/home/andes/skripsi/kodingan2/runs/train/exp12/weights/best.pt')  # Ganti dengan path ke model best.pt Anda

# Baca video
video_path = '/home/andes/skripsi/kodingan2/Test-Vid/17-1.mp4'  # Ganti dengan path video yang ingin diuji
cap = cv2.VideoCapture(video_path)

# Mendapatkan ukuran frame untuk output video
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS))

# Output video
out = cv2.VideoWriter('counting.mp4', cv2.VideoWriter_fourcc(*'mp4v'), fps, (frame_width, frame_height))

# Variabel untuk menghitung total jumlah bibit dan jumlah frame
total_count = 0
frame_count = 0

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Melakukan inferensi pada setiap frame
    results = model.predict(frame, save=False)  # Prediksi tanpa menyimpan otomatis
    
    # Hitung jumlah deteksi (jumlah bounding box yang ditemukan)
    count = 0  # Inisialisasi perhitungan jumlah bibit ikan
    for result in results:
        count = len(result.boxes)  # Hitung jumlah box deteksi

        # Gambar bounding box tanpa label
        for box in result.boxes:
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)  # Bounding box hijau

    # Tambahkan ke total_count dan tambahkan frame_count
    total_count += count
    frame_count += 1

    # Tampilkan jumlah bibit ikan yang terdeteksi di layar
    text = f'Total number of seeds detected: {count}'
    
    # Lokasi teks
    org = (20, 40)
    
    # Tambahkan shadow (teks hitam di belakang)
    cv2.putText(frame, text, (org[0] + 2, org[1] + 2), cv2.FONT_HERSHEY_SIMPLEX, 1.2, (0, 0, 0), 4)  # Bayangan hitam, tebal 4 px
    
    # Tambahkan teks putih di atas shadow (Bold dengan ketebalan lebih)
    cv2.putText(frame, text, org, cv2.FONT_HERSHEY_SIMPLEX, 1.2, (0, 0, 255), 3)  # Teks merah, tebal 3 px

    # Simpan frame dengan bounding box dan jumlah deteksi
    out.write(frame)

    # Tampilkan hasil secara langsung (opsional)
    cv2.imshow('YOLOv10 Inference', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Hitung rata-rata jumlah bibit per frame
average_count = total_count / frame_count if frame_count > 0 else 0

# Menampilkan jendela baru dengan rata-rata jumlah bibit
new_window_width = 1200
new_window_height = 400
final_frame = np.zeros((new_window_height, new_window_width, 3), dtype=np.uint8)

# Teks rata-rata bibit yang terdeteksi
final_text = f'Average seeds detected: {average_count:.2f}'

# Ukuran teks
(text_width, text_height), _ = cv2.getTextSize(final_text, cv2.FONT_HERSHEY_SIMPLEX, 1.5, 3)

# Posisi teks di tengah frame
text_x = (new_window_width - text_width) // 2
text_y = (new_window_height + text_height) // 2

# Tambahkan shadow (teks hitam di belakang)
cv2.putText(final_frame, final_text, (text_x + 2, text_y + 2), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (0, 0, 0), 5)

# Tambahkan teks merah di atas shadow
cv2.putText(final_frame, final_text, (text_x, text_y), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (0, 0, 255), 3)

# Tampilkan jendela baru dengan rata-rata bibit terdeteksi
cv2.imshow('Average Seeds Detected', final_frame)
cv2.waitKey(0)

# Melepaskan resource
cap.release()
out.release()
cv2.destroyAllWindows()



0: 640x608 22 BibitNilas, 10.4ms
Speed: 2.9ms preprocess, 10.4ms inference, 0.4ms postprocess per image at shape (1, 3, 640, 608)

0: 640x608 19 BibitNilas, 22.9ms
Speed: 4.2ms preprocess, 22.9ms inference, 1.8ms postprocess per image at shape (1, 3, 640, 608)

0: 640x608 18 BibitNilas, 9.7ms
Speed: 3.0ms preprocess, 9.7ms inference, 0.5ms postprocess per image at shape (1, 3, 640, 608)

0: 640x608 18 BibitNilas, 30.5ms
Speed: 4.2ms preprocess, 30.5ms inference, 1.6ms postprocess per image at shape (1, 3, 640, 608)

0: 640x608 17 BibitNilas, 10.5ms
Speed: 3.0ms preprocess, 10.5ms inference, 0.5ms postprocess per image at shape (1, 3, 640, 608)

0: 640x608 19 BibitNilas, 27.7ms
Speed: 4.2ms preprocess, 27.7ms inference, 1.3ms postprocess per image at shape (1, 3, 640, 608)

0: 640x608 19 BibitNilas, 18.5ms
Speed: 3.5ms preprocess, 18.5ms inference, 1.4ms postprocess per image at shape (1, 3, 640, 608)

0: 640x608 18 BibitNilas, 13.0ms
Speed: 4.4ms preprocess, 13.0ms inference, 0.5ms po

In [2]:
import numpy as np
import math

# Fungsi untuk menghitung RMSE
def calculate_rmse(ground_truth, predictions):
    # Pastikan panjang array ground truth dan prediksi sama
    if len(ground_truth) != len(predictions):
        raise ValueError("Ground truth dan prediksi harus memiliki panjang yang sama.")
    
    # Hitung error kuadrat
    squared_errors = [(gt - pred) ** 2 for gt, pred in zip(ground_truth, predictions)]
    
    # Hitung mean dari error kuadrat
    mean_squared_error = np.mean(squared_errors)
    
    # Ambil akar kuadrat dari MSE untuk mendapatkan RMSE
    rmse = math.sqrt(mean_squared_error)
    return rmse

# Contoh penggunaan
# Misalkan ground truth (jumlah bibit sebenarnya) dan prediksi (dari model YOLO) adalah sebagai berikut:
ground_truth_counts = [17]  # Sesuaikan dengan ground truth yang Anda miliki
predicted_counts = [18.6]     # Sesuaikan dengan prediksi dari model YOLO

# Hitung RMSE
rmse = calculate_rmse(ground_truth_counts, predicted_counts)

print(f"RMSE: {rmse:.2f}")


RMSE: 1.60
